In [ ]:
import json
import os
import sys
import fnmatch


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict

In [ ]:
df = pd.read_csv('500_year.csv')#big_stat
#df = df[(df.G*df.MP > 850)]
print(len(df.columns))
#df.columns = dfc
df.shape
#df = df.sample(10000)


In [ ]:
y = df.iloc[:,-15:]
X = df.iloc[:,11:-17]
#y = y[(X['AST%'] >0) & (X['AST%'] < 100)]
print(X.shape)
#X = X[(X['AST%'] >0) & (X['AST%'] < 100)]
print(X.shape)

X['MP'] = df.MP
X['Hgt'] = df['Hgt']

stat_list = ['FG','FGA','3P',"3PA",'FT','FTA',\
             'ORB','DRB','TRB','AST','TOV','STL',"BLK",\
             'PF','PTS']
for name in stat_list:
    den = np.maximum(1,df.MP)
    X[name + 'p36'] = 36* df[name]/den
    X[name + 'p100'] = X[name + 'p36']*4/3

adv_stat_list = ['OWS','DWS','WS']
for name in adv_stat_list:
    
    den = np.maximum(1,df.MP*df['G'])
    #if name in ['OWS','DWS']:
    #    den = den*df['G']
    #X[name] = df[name]
    X[name + 'p36_G72'] = 72*36* df[name]/den
    

X['3PtP'] = (2/(1+np.exp(-X['3PAp100']))-1)*X['3P%']/100
X['Creation'] = X['ASTp100']*0.1843+(X['PTSp100']+X['TOVp100'])*0.0969-2.3021*X['3PtP']+0.0582*(X['ASTp100']*(X['PTSp100']+X['TOVp100'])*X['3PtP'] )-1.1942
X['Load'] = (X['ASTp100']-(0.38*X['Creation'])*0.75)+X['FGAp100']+X['FTAp100']*0.44+X['Creation']+X['TOVp100']
X['cTOV'] = X['TOVp100']/X['Load']
#X['DPM'] = X['Blkp100']*0.802+X['DRBp100']*0.42-4.7-0.07551*X['PFp100']+1.597019*X['STLp100']-0.26385*X['TOVp100']
#X['OPM'] = -8.57647+0.6111*X['PTSp100']-0.33918*(0.44*X['FTAp100']+X['FGAp100'])+0.440814*X['FTAp100']+0.379745*X['3PAp100']+0.634044*X['ASTp100']+0.77827*X['ORBp100']-1.08855*X['TOVp100']+0.26262*X['STLp100']
#X['BPM'] = X['OPM'] + X['DPM']
X['Age'] = df['Age']
X['DRE'] = -8.4+ 0.8*X['PTSp100'] -0.7*(X['FGAp100']-X['3PAp100']) -0.6*X['3PAp100'] - 0.2*X['FTp100'] + 0.1*X['ORBp100'] + 0.4*X['DRBp100'] + 0.5*X['ASTp100'] + 1.7*X['STLp100'] + 0.8*X['BLKp100'] - 1.4*X['TOVp100'] - 0.1*X['PFp100']
for name in [_ for _ in X.columns if 'FG' in _]:
    name3 = name.replace('FG','3P')
    if '%' in name or name3 not in list(X.columns):
        continue
    nname = name.replace('FG','2P')
    X[nname] = X[name] - X[name3]
    print(name)

#else:
#X['PER'] = df['PER']

    #X['DREO'] = X['PTSp100'] + .2*X['TRBp100'] + .5*X['ASTp100'] - .9*X['FGAp100'] - .35*X['FTAp100']-6.23
#X['PassP'] = ((X['ASTp100']-(0.38*X['Creation']))*0.752+ X['Creation'] + X['TOVp100']) ** 0.67
#'OPM','DPM','cTOV','Load'#stat_list[:-2]+
#'PER','FG%','DREO'



X = X[[_ for _ in X.columns if '%' in _ or _[-1]=='r' or 'p36' in _ or _ in (['MP','TS%','FT%','3P%','OSPM','PER','DRE','FT%','OPM','BPM','DPM','Creation','cTOV','Load','Age','MP'])]]
if False:
    X = X[[_ for _ in X.columns if not '3P' in _]]
    X = X[[_ for _ in X.columns if not 'Rim' in _]]
    X = X[[_ for _ in X.columns if not 'Post' in _]]
    X = X[[_ for _ in X.columns if not 'Mid' in _]]
    X = X[[_ for _ in X.columns if not 'TOV' in _]]
    X = X[[_ for _ in X.columns if not 'Blk' in _]]
    X = X[[_ for _ in X.columns if not 'ORB' in _]]
    X = X[[_ for _ in X.columns if not 'DRB' in _]]
    X = X[[_ for _ in X.columns if not 'STL' in _]]
    X = X[[_ for _ in X.columns if not 'DPM' in _]]
    X = X[[_ for _ in X.columns if not 'OPM' in _]]
    X = X[[_ for _ in X.columns if not 'BPM' in _]]
    X = X[[_ for _ in X.columns if not '+/-' in _]]
else:
    X = X[[_ for _ in X.columns if not 'FG%' in _]]
    X = X[[_ for _ in X.columns if not '2P%' in _]]
    X = X[[_ for _ in X.columns if _ not in ['2Pp36','2PAp36','3PAr']]]
    X = X[[_ for _ in X.columns if not 'BPM' in _]]
    X = X[[_ for _ in X.columns if not 'TRB' in _]]
    X = X[[_ for _ in X.columns if not 'DRB%' in _]]
    X = X[[_ for _ in X.columns if not 'ORB%' in _]]
    X = X[[_ for _ in X.columns if _ not in ['AST%','STL%','BLK%','TOV%','USG%','TS%']]]

#X['OBPM'] = df['OBPM']
#X['DBPM'] = df['DBPM']
#X = X[[_ for _ in X.columns if not '3PAr' in _]]

replacement_filter = (df.Salary > 0.5) & (df.Salary < 1.0)
replacement_player_mean_bs = X[replacement_filter].mean()
replacement_player_std_bs = X[replacement_filter].std()
replacement_player_cov_bs = X[replacement_filter].cov()

replacement_player_mean_r = y[replacement_filter].mean()
replacement_player_std_r = y[replacement_filter].std()
replacement_player_cov_r = y[replacement_filter].cov()


replacement_player_mean_r

X.columns


In [ ]:
ovr_weights =  {'diq': 0.093,
 'dnk': 0.0424,
 'drb': 0.0968,
 'end': 0.0075,
 '2pt': -0.0093,
 'ft.1': 0.049,
 'hgt': 0.225,
 'ins': -0.0143,
 'jmp': 0.0505,
 'oiq': 0.0971,
 'pss': 0.0657,
 'reb': 0.0534,
 'spd': 0.156,
 'str': 0.0962,
 '3pt': 0.105}
ovr_v = np.array([ovr_weights[c.lower()] for c in y.columns])

In [ ]:
#np.round(replacement_player_std_r).astype(np.int)
#_ =  plt.hist(X['OSPM'],150)
#X['OSPM'].mean()

In [ ]:
from sklearn import neural_network
from sklearn import linear_model
from sklearn import preprocessing
from sklearn import feature_selection, model_selection
from sklearn import  multioutput
from sklearn import ensemble
from sklearn import svm

In [ ]:
#np.array([cc.alpha_ for cc in clf.estimators_]).mean()

In [ ]:
fexp = preprocessing.PolynomialFeatures(degree=2,interaction_only=False)
scalerX = preprocessing.RobustScaler()
scalery = preprocessing.StandardScaler()

prescale_X = fexp.fit_transform(X)
prescale_X = scalerX.fit_transform(prescale_X)

prescale_y = scalery.fit_transform(y)
X_train, X_test, y_train, y_test = model_selection.train_test_split(prescale_X, prescale_y, test_size=0.25, random_state=42)

In [ ]:
alphas2 = [
 0.0012093412968659217,
 0.0021644471822303744,
 0.002476479841448311,
 0.003984192886046182,
 0.0033815611873227974,
 0.0059244781808397125,
 0.006696280944494202]
alphas2 = np.linspace(0.001,0.007,8)

In [ ]:
alpha_max = (np.sqrt(np.sum(prescale_X ** 2, axis=1)).max() /
             (prescale_X.shape[0]))


In [ ]:
#alpha_max = 1.0
eps = 1e-6
n_alphas = 10
alphas = np.logspace(np.log10(alpha_max * eps), np.log10(alpha_max),
                       num=n_alphas)[::-1]

In [ ]:
#prescale_X = np.append(prescale_X,np.zeros_like(prescale_X[:,0:1]),1)
X_train2, X_test2, y_train2, y_test2 = model_selection.train_test_split(prescale_X, prescale_y, test_size=0.15, random_state=48)

In [ ]:
X_test2.shape[0],y_test2.shape

In [ ]:
A = prescale_X.T @ prescale_X + np.exp(-8)*np.identity(prescale_X.shape[1])
Ainv = np.linalg.pinv(A)
coef = (Ainv @ prescale_X.T) @ prescale_y
coef

In [ ]:
trials = 1
ts = []
for i in range(trials):
    #clf = neural_network.MLPRegressor((36,5,24,36),'tanh',solver='adam',max_iter=1000)
    #clf = neural_network.MLPRegressor((),'identity',solver='lbfgs',alpha=5e2,tol=1e-9)
    #clf = multioutput.MultiOutputRegressor(linear_model.SGDRegressor(penalty='l2',verbose=True))
    #clf = multioutput.MultiOutputRegressor(linear_model.ElasticNetCV(cv=3,n_alphas=10,max_iter=2500))
    #clf = multioutput.MultiOutputRegressor(linear_model.RidgeCV(cv=3))
    clf = multioutput.MultiOutputRegressor(linear_model.ElasticNet(alpha=0.001, max_iter=5000))

    #clf = multioutput.MultiOutputRegressor(linear_model.ElasticNet(alpha=5e-3,l1_ratio=0.5))

    #clf = ensemble.ExtraTreesRegressor(8,criterion='mae',max_depth=3,verbose=1)
    #clf = multioutput.MultiOutputRegressor(svm.SVR())
    #clf.fit(X_train,y_train)
    #print(clf.score(X_train,y_train),clf.score(X_test,y_test))
    clf.fit(prescale_X,prescale_y,np.array(df.G*df.MP))
    yt = scalery.inverse_transform(clf.predict(prescale_X))
    err = np.linalg.norm(yt-y)
    ts.append((err,clf))

In [ ]:
#import pickle
#clf = pickle.load(open('model_dump.pkl','rb'))


In [ ]:
import pickle
pickle.dump(clf,open('model_dump_weighted_play.pkl','wb'))


In [ ]:
ts = sorted(ts)[::1] # why not the biggest error
print(ts[0][0])
clf = ts[0][1] # 7179 for RidgeCV # 7321

In [ ]:
X.columns

In [ ]:
col_names = X.columns
col_names = fexp.get_feature_names(X.columns)

for i,c in enumerate(y.columns):
    coeffs = clf.estimators_[i].coef_ 
    v = np.argsort(abs(coeffs))[::-1]
    print(c)
    coeffs2 = [(coeffs[i2],col_names[i2]) for i2 in v[:80]]
    #for v,n in sorted(coeffs2,reverse=True):
    #    print('{:.2f} * {} + '.format(v,n),end='')
    print('| Variable | Coeff |')
    print('|----------|-------|')
    for v,n in sorted(coeffs2,reverse=True):
        print('|{:25s}|{:.2f}|'.format(n,v))
    #for v,n in sorted(coeffs2,reverse=True):
    #    print('\t{:25s}\t{:.2f}'.format(n,v))


In [ ]:
base = json.load(open('BBGM_ASG_2022_2022_preseason.json','rb'))

In [ ]:
GEN_YEAR = 2022
adv = pd.read_csv('{}_adv.csv'.format(GEN_YEAR))
poss = pd.read_csv('{}_poss.csv'.format(GEN_YEAR))

adv['srID'] = adv.Player.map(lambda x: x.split('\\')[1])
poss['srID'] = poss.Player.map(lambda x: x.split('\\')[1])

multi = set(adv[adv.Tm == 'TOT'].srID)

adv1 = adv[adv.srID.map(lambda x: x in multi) & (adv.Tm == 'TOT') ]
adv2 = adv[~adv.srID.map(lambda x: x in multi)]
adv = pd.concat([adv1,adv2]).set_index('srID')

multi = set(poss[poss.Tm == 'TOT'].srID)
poss1 = poss[poss.srID.map(lambda x: x in multi) & (poss.Tm == 'TOT') ]
poss2 = poss[~poss.srID.map(lambda x: x in multi)]
poss = pd.concat([poss1,poss2]).set_index('srID')

df_nba = pd.merge(poss,adv,how="inner", left_index=True, right_index=True)
df_nba = df_nba.dropna(1,how='all')

In [ ]:
heights = {}
for p in base['players']:
    if 'srID' not in p:
        continue
    heights[p['srID']] = p['hgt']
df_nba['height'] = pd.Series(heights)
df_nba = df_nba.dropna()
df_nba.columns = [_.replace('%','p') for _ in df_nba.columns]
df_nba['height'] = np.clip((df_nba.height-66)*3.70,0,100)

In [ ]:
RATE_SCALES = ['FG%','TS%','3P%','FT%','3PAr','2P%']

In [ ]:
player_vectors = []
player_scales = []
player_heights = []
player_names = []
for idx,row in df_nba.iterrows():
    d = dict(row)
    
    d['Hgt'] = row['height']
    d['Age']= row['Age_x']

    d['DRE'] = -8.4+ 0.8*d['PTS'] -0.7*(d['FGA']-d['3PA']) -0.6*d['3PA'] - 0.2*d['FT'] + 0.1*d['ORB'] + 0.4*d['DRB'] + 0.5*d['AST'] + 1.7*d['STL'] + 0.8*d['BLK'] - 1.4*d['TOV'] - 0.1*d['PF']
    d['3PtP'] = (2/(1+np.exp(-d['3PA']))-1)*d['3Pp']
    d['Creation'] = d['AST']*0.1843+(d['PTS']+d['TOV'])*0.0969-2.3021*d['3PtP']+0.0582*(d['AST']*(d['PTS']+d['TOV'])*d['3PtP'] )-1.1942
    d['Load'] = (d['AST']-(0.38*d['Creation'])*0.75)+d['FGA']+d['FTA']*0.44+d['Creation']+d['TOV']
    d['cTOV'] = d['TOV']/d['Load']
    #d['DPM'] = d['BLK']*0.802+d['DRB']*0.42-4.7-0.07551*d['PF']+1.597019*d['STL']-0.26385*d['TOV']
    #d['OPM'] = -8.57647+0.6111*d['PTS']-0.33918*(0.44*d['FTA']+d['FGA'])+0.440814*d['FTA']+0.379745*d['3PA']+0.634044*d['AST']+0.77827*d['ORB']-1.08855*d['TOV']+0.26262*d['STL']
    #d['BPM'] = d['OPM']+d['DPM']

    MPo = np.maximum(1,np.nan_to_num(d['MP_x']))
    MP = MPo
    for statn in adv_stat_list:
        d[statn+'p36_G72'] = 72*36*float(d[statn])/MPo#72*36*d[statn]/MPo
    for s in RATE_SCALES:
        d[s.replace('%','p')] *= 100
    
    d['MP'] = row['MP_x']/row['G_x']
    d['Blk'] = d['BLK']

    player_vectors.append([d[stat.replace('p36','').replace('%','p') if 'G72' not in stat else stat] for stat in X.columns])
    player_scales.append(MP)
    player_heights.append(d['Hgt'])
    player_names.append(idx)
X.columns

In [ ]:
pd.DataFrame(player_vectors,columns=X.columns)#.iloc[player_names.index('Collin Sexton')]
#player_scales[93]

In [ ]:
Xn = np.nan_to_num(np.array(player_vectors))
Xn = np.nan_to_num(Xn)
first_n = Xn.shape[0]
#Xn.shape,Xn_s.shape,prescale_X.shape,fexp.n_input_features_,X.shape

In [ ]:
c = np.array(player_scales)
print(c.max())
scaling = (c.max()/3028) * 256 # adjust for era
c = np.array(player_scales).reshape((-1,1))
c = np.tanh(c/scaling) # basically 1 by 512 MP


In [ ]:
plt.plot(np.linspace(0,250),np.tanh(np.linspace(0,250) * (256/3028)))

In [ ]:
bins = [-100]  + list(range(30,85)) + [200]
base_hist = np.histogram((y*ovr_v).sum(1),bins,density=True)[0] + 1e-9

def just_a_thing(qrange):
    #Xn = np.nan_to_num(np.array(player_vectors))
    # tuned this to get roughly 8-12 players at 70 or above. Which seemed like normal for a league
    scalerX2 = preprocessing.RobustScaler(quantile_range=tuple(qrange))

    #scalerX2 = scalerX
    Xn_s = fexp.transform(np.nan_to_num(Xn))
    Xsub = Xn_s[:first_n]
    Xsub = Xsub[(c.ravel()[:first_n] > 0.15)]

    scalerX2.fit(Xsub)
    Xn_fs =scalerX2.transform(Xn_s)


    predict =clf.predict(Xn_fs) #  Xn_fs @ coef#
    ratings = np.nan_to_num(scalery.inverse_transform(predict))
    #ratings[:,0] = Xn[:,list(X.columns).index('Hgt')]
    HGT_PRED = 0.35
    #if 'Hgt' in list(X.columns):
    #ratings[:,0] = HGT_PRED*ratings[:,0] + (1-HGT_PRED)* np.array(player_heights)
    ratings[:,0] = HGT_PRED*np.array(player_heights) + (1-HGT_PRED)*np.maximum(ratings[:,0],np.array(player_heights))
    # now we can sample directly from ratings

    # if we want to scale players down based on minutes played to replacement level
    if True:
        ratings[:predict.shape[0]] = ratings[:predict.shape[0]]*c + (1-c)*np.repeat(np.array(replacement_player_mean_r).reshape((-1,1)),predict.shape[0],1).T
    
    est_hist = np.histogram((ratings*ovr_v).sum(1),bins,density=True)[0] + 1e-9
    
    kl1 = est_hist * np.log(est_hist/base_hist)
    kl2 = base_hist * np.log(base_hist/est_hist)
    return kl1.sum() + kl1.sum()
qrange_init = [26.31838933, 69.76304586]
just_a_thing(qrange_init)

In [ ]:
import cma
es = cma.CMAEvolutionStrategy(qrange_init,2)
es.optimize(just_a_thing)
qrange = es.best.x

In [ ]:
#Xn = np.nan_to_num(np.array(player_vectors))
# tuned this to get roughly 8-12 players at 70 or above. Which seemed like normal for a league
scalerX2 = preprocessing.RobustScaler(quantile_range=tuple(qrange))

#scalerX2 = scalerX
Xn_s = fexp.transform(np.nan_to_num(Xn))
Xsub = Xn_s[:first_n]
Xsub = Xsub[(c.ravel()[:first_n] > 0.15)]

scalerX2.fit(Xsub)
Xn_fs =scalerX2.transform(Xn_s)


predict =clf.predict(Xn_fs) #  Xn_fs @ coef#
ratings = np.nan_to_num(scalery.inverse_transform(predict))
#ratings[:,0] = Xn[:,list(X.columns).index('Hgt')]
HGT_PRED = 0.35
#if 'Hgt' in list(X.columns):
#ratings[:,0] = HGT_PRED*ratings[:,0] + (1-HGT_PRED)* np.array(player_heights)
ratings[:,0] = HGT_PRED*np.array(player_heights) + (1-HGT_PRED)*np.maximum(ratings[:,0],np.array(player_heights))
# now we can sample directly from ratings

# if we want to scale players down based on minutes played to replacement level
if True:
    ratings[:predict.shape[0]] = ratings[:predict.shape[0]]*c + (1-c)*np.repeat(np.array(replacement_player_mean_r).reshape((-1,1)),predict.shape[0],1).T


In [ ]:
plt.hist((ratings*ovr_v).sum(1),bins,alpha=0.5,density=True)
plt.hist((y*ovr_v).sum(1),bins,alpha=0.5,density=True)
plt.xlim(35,80)

In [ ]:
Xn_fs[93,col_names.index('3PAp36^2')]
#for k,v in zip(y.columns,ratings[93]):
#    print(k,v)

In [ ]:
#for n,v in zip(X.columns,Xn_s[player_names.index('Draymond Green')]):
#    print(n,v)

In [ ]:
HGT_PRED = 0.5
player_heights = np.array(player_heights)

def normalize(fro,to):
    tm,ts = to.mean(),to.std()
    fm,fs = fro.mean(),fro.std()
    z = (fro-fm)/fs
    return z*ts + tm
nr = normalize(ratings[:,0],player_heights)
tmp = np.clip(HGT_PRED*np.array(player_heights) + (1-HGT_PRED)*np.maximum(nr,player_heights),0,100)

if True:
    plt.figure(figsize=(12,4))
    plt.subplot(1,3,1)
    _ = plt.hist(player_heights,15,alpha=0.8,density=True,label='NBA')
    _ = plt.hist(np.clip(ratings[:,0],0,100),15,alpha=0.5,density=True,label='Model')
    plt.xlim(0,100)
    plt.ylim(0,0.05)
    plt.legend()
    plt.subplot(1,3,2)
    _ = plt.hist(player_heights,15,alpha=0.8,density=True,label='NBA')
    plt.hist(tmp,15,alpha=0.5,density=True,label='Blended',color='#2ca02c')
    plt.xlim(0,100)
    plt.ylim(0,0.05)
    plt.legend()
    plt.subplot(1,3,3)
    _ = plt.hist(df['Hgt'],15,alpha=0.8,density=True,label='BBGM',color='#7f7f7f')
    plt.hist(tmp,15,alpha=0.5,density=True,label='Blended',color='#2ca02c')
    plt.xlim(0,100)
    plt.ylim(0,0.05)
    plt.legend()
    plt.savefig('test.png',edgecolor='w',facecolor='w')

In [ ]:
plt.hist(player_heights,50)

In [ ]:

#ratings[:,0]
#table_columns['per_minute']#,table_columns['advanced'].index('0-3')
#for i,t in enumerate(player_stats[name]['advanced'][0]):
#    print(i,t)

In [ ]:
#player_vectors[player_names.index('Joel Embiid')][list(X.columns).index('OWSp36')],X.columns[list(X.columns).index('OWSp36')]
#player_stats[2019]['Joel Embiid']['advanced'][0][12],player_stats[2019]['Joel Embiid']['per_game'][0][3]

In [ ]:
#_ = plt.hist(Xn[:,list(X.columns).index('FTp36')],150,density=True,alpha=0.5,label='NBA')
#_ = plt.hist(X['FTp36'],150,alpha=0.5,density=True,label='BBGM')
#plt.legend()
#plt.figure()
_ = plt.hist(player_heights,15,density=True,alpha=0.5,label='NBA')
_ = plt.hist(ratings[:,0],15,alpha=0.5,density=True,label='BBGM')
plt.title('height distribution')
plt.legend()
plt.xlim(0,100)

In [ ]:
X.mean(0)-Xn.mean(0)

In [ ]:
Xn.shape,X.shape

In [ ]:
import random
def load_roster(filename):
    base = json.load(open(filename,'rb'))
    players = base['players']
    return players,base
playersOG,base = load_roster('BBGM_ASG_2022_2022_preseason.json')
abbrev_to_tid = {_['abbrev']:_['tid'] for _ in base['teams']}
abbrev_to_tid['BRK'] = abbrev_to_tid['BKN']
abbrev_to_tid['CHO'] = abbrev_to_tid['CHA']
abbrev_to_tid['GSW'] = abbrev_to_tid['GS']
abbrev_to_tid['NOP'] = abbrev_to_tid['NOL']
abbrev_to_tid['NYK'] = abbrev_to_tid['NYC']
abbrev_to_tid['SAS'] = abbrev_to_tid['SA']


y_keys = [_.lower() for _ in y.columns]

y_map = { 'hgt': 'hgt',
   'stre': 'str',
   'spd': 'spd',
   'jmp': 'jmp',
   'endu': 'end',
   'ins': 'ins',
   'dnk': 'dnk',
   'ft': 'ft.1',
   'fg': '2pt',
   'tp': '3pt',
   'diq': 'diq',
   'oiq': 'oiq',
   'drb': 'drb',
   'pss': 'pss',
   'reb': 'reb' }

In [ ]:
[_ for _ in base['players'] if 'srID' in _ and _['srID'] == 'doncilu01']
#base['players'][0]

In [ ]:
r_lookup = {}
r_min = {}
for name,r,mp in zip(player_names,ratings,np.squeeze(c)):
    r_vec = {k: r[y_keys.index(km)] for k,km in y_map.items()}
    r_vec = {k: int(np.clip(v,0,100)) for k,v in r_vec.items()}
    r_vec['season'] = GEN_YEAR
    r_lookup[name] = r_vec
    r_min[name] = mp

In [ ]:
filt_cols =['value','valueNoPot','valueFuzz','valueNoPotFuzz','fuzz']
for i in range(len(base['players'])):
    if 'srID' not in base['players'][i]:
        continue
    srID = base['players'][i]['srID']
    if srID not in r_lookup:
        continue
    if r_min[srID] < 0.8:
        continue
    base['players'][i]['ratings'][-1] = r_lookup[srID]
    for idx in range(len(base['players'][i]['ratings'])):
        for thing in filt_cols:
            rt = base['players'][i]['ratings'][idx]
            if thing in rt:
                del base['players'][i]['ratings'][idx][thing]
    for thing in filt_cols:
        if thing in base['players'][i]:
            del base['players'][i][thing]
    if srID == 'hardeja01':
        print(base['players'][i])

In [ ]:
with open('auto_roster_{}.json'.format(GEN_YEAR),'wt') as fp:
    json.dump(base,fp, sort_keys=True)

In [ ]:
player_names

In [ ]:
r_lookup['hardeja01']